In [1]:
import dlib
import numpy as np
from os import path
from PIL import Image
from glob import glob
from skimage.color import rgb2gray

import matplotlib.pyplot as plt

In [2]:
# https://github.com/jrosebr1/imutils/blob/master/imutils/face_utils/helpers.py

LAND_MARKS = {"right_eye": (36, 42),
            "left_eye": (42, 48)}

In [3]:
def new_filename(file):
    return path.join('Aligned', file.replace(path.dirname(file) + '/', ''))

def get_date_taken(image):
    exif = image._getexif()
    if exif is not None:
        return exif[36867]
    return None

def reshape(image):
    width, height = image.size
    ratio = min(MAX_WIDTH / width, MAX_HEIGHT / height)
    if ratio != 1.0:
        simage = image.copy()
        simage.thumbnail([width * ratio, height * ratio], Image.ANTIALIAS)
        return simage
    else:
        return image

def reshape_to_face(image, width, height):
    w, h = image.size
    ratio = min(FACE_WIDTH / width, FACE_HEIGHT / height)
    if ratio != 1.0:
        simage = image.copy()
        simage.thumbnail([w * ratio, h * ratio], Image.ANTIALIAS)
        return simage
    else:
        return image
    
def points_to_xy(points):
    return np.array([(point.x, point.y) for point in points])

def align_image(image):
    matrix = (255 * rgb2gray(np.array(image))).astype('uint8')
    ans = detector(matrix, 2)
    if len(ans):
        ans = ans[0]
        points = predictor(matrix, ans).parts()
        
        width = ans.width()
        height = ans.height()
        
        a, b = LAND_MARKS['right_eye']
        right = points_to_xy(points[a:b]).mean(axis=0).astype(int)

        a, b = LAND_MARKS['left_eye']
        left = points_to_xy(points[a:b]).mean(axis=0).astype(int)

        delta = right - left
        angle = np.degrees(np.arctan2(delta[1], delta[0])) - 180

        middle = (0.5 * (right + left)).astype(int)

        x = middle[0] - image.size[0] // 2
        y = middle[1] - image.size[1] // 2

        new_image = image.transform(image.size, Image.AFFINE, (1, 0, x, 0, 1, y))
        new_image = new_image.rotate(angle)
        
        new_image = reshape_to_face(new_image, width, height)
        
        return middle, new_image
    return None, None


def center_crop(img):
    im = img.copy()
    width, height = im.size   # Get dimensions

    left = (width - MAX_WIDTH)/2
    top = (height - MAX_HEIGHT)/2
    right = (width + MAX_WIDTH)/2
    bottom = (height + MAX_HEIGHT)/2

    # Crop the center of the image
    return im.crop((left, top, right, bottom))
    
#     if type(img) is Image.Image:
#         img = np.array(img)
    
#     width, height = img.shape[:2]
    
#     left = int(np.ceil((width - new_width) / 2))
#     right = width - int(np.floor((width - new_width) / 2))

#     top = int(np.ceil((height - new_height) / 2))
#     bottom = height - int(np.floor((height - new_height) / 2))
        
#     center_cropped_img = img[top:bottom, left:right]
    
#     return Image.fromarray(center_cropped_img)

In [4]:
FACE_WIDTH = 200
FACE_HEIGHT = 300

MAX_WIDTH = 1280
MAX_HEIGHT = 720

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [5]:
files = glob("Laura/*")

In [6]:
for i, file in enumerate(files):
    image = reshape(Image.open(file))
    middle, new_image = align_image(image)

    if new_image is not None:
        new_image = center_crop(new_image)
        file = new_filename(file)
        new_image.save(file)
        print("%d/%d:" % (i + 1, len(files)), file)
#         break

1/447: Aligned/IMG-20181128-WA0008.jpeg
2/447: Aligned/IMG-20181219-WA0023.jpg
4/447: Aligned/IMG-20190731-WA0005.jpg
5/447: Aligned/IMG-20181218-WA0049.jpg
6/447: Aligned/IMG-20190526-WA0009.jpg
7/447: Aligned/IMG-20190202-WA0000.jpeg
8/447: Aligned/IMG-20190214-WA0013.jpg
9/447: Aligned/IMG-20190202-WA0011.jpeg
10/447: Aligned/IMG-20181215-WA0016.jpg
12/447: Aligned/IMG-20181219-WA0040.jpg
13/447: Aligned/IMG-20190826-WA0017.jpg
14/447: Aligned/IMG-20190816-WA0009.jpg
15/447: Aligned/IMG-20190312-WA0000.jpg
16/447: Aligned/IMG-20190804-WA0008.jpg
17/447: Aligned/IMG-20190522-WA0052.jpg
18/447: Aligned/IMG_20190817_141240.jpg
19/447: Aligned/IMG-20181219-WA0029.jpg
20/447: Aligned/IMG_20190819_173841.jpg
21/447: Aligned/IMG-20190826-WA0014.jpg
22/447: Aligned/IMG-20181109-WA0012.jpeg
23/447: Aligned/IMG-20190823-WA0001.jpeg
25/447: Aligned/IMG-20181216-WA0046.jpeg
26/447: Aligned/IMG-20190831-WA0005.jpg
27/447: Aligned/IMG-20181106-WA0008.jpeg
28/447: Aligned/IMG-20181219-WA0038.jpg
2

233/447: Aligned/IMG-20190804-WA0028.jpg
234/447: Aligned/IMG-20190416-WA0001.jpg
236/447: Aligned/IMG-20181219-WA0028.jpg
237/447: Aligned/IMG-20190826-WA0026.jpg
238/447: Aligned/IMG-20190816-WA0010.jpg
239/447: Aligned/IMG-20190630-WA0313.jpeg
240/447: Aligned/IMG-20190804-WA0029.jpg
241/447: Aligned/IMG-20190407-WA0040.jpg
242/447: Aligned/IMG-20190414-WA0027.jpg
243/447: Aligned/IMG-20190414-WA0019.jpg
244/447: Aligned/IMG-20190414-WA0033.jpg
245/447: Aligned/IMG-20190708-WA0007.jpg
246/447: Aligned/IMG-20190609-WA0015.jpg
247/447: Aligned/IMG-20190615-WA0036.jpg
248/447: Aligned/IMG-20190113-WA0005.jpg
249/447: Aligned/IMG-20190522-WA0037.jpg
250/447: Aligned/IMG-20190407-WA0047.jpg
251/447: Aligned/IMG-20190511-WA0040.jpg
252/447: Aligned/IMG-20190407-WA0038.jpg
253/447: Aligned/IMG-20190210-WA0006.jpg
255/447: Aligned/IMG-20181108-WA0021.jpeg
256/447: Aligned/IMG-20190826-WA0036.jpg
257/447: Aligned/IMG-20190311-WA0012.jpg
258/447: Aligned/IMG-20190826-WA0020.jpg
262/447: Align